# Preprocess data and extract features.

In [1]:
%env PYTHONHASHSEED=13
%load_ext autoreload
%autoreload 2

env: PYTHONHASHSEED=13


In [2]:
# If you need fully deterministic results between runs, set the following environment value prior to launching jupyter.
# Instructions can be found in HOW-TO-ENVIRONMENT.md.
# See comment in sherlock.features.paragraph_vectors.infer_paragraph_embeddings_features for more info.
%env PYTHONHASHSEED

'13'

In [3]:
from datetime import datetime
import os
import sys
import time

import numpy as np
import pandas as pd

from sherlock import helpers
from sherlock.functional import extract_features_to_csv
from sherlock.features.paragraph_vectors import initialise_pretrained_model, initialise_nltk
from sherlock.features.preprocessing import (
    extract_features,
    convert_string_lists_to_lists,
    prepare_feature_extraction,
    load_parquet_values,
)
from sherlock.features.word_embeddings import initialise_word_embeddings

In [4]:
print(f'Started at {datetime.now()}.')

Started at 2022-11-28 06:26:32.874970.


## Download data
This will download the raw values and preprocessed files, the corresponding labels as well as a few other supporting files:
- `download_data()` will download 3.6GB of data for preprocessing and model training into the `data/` directory.
- `prepare_feature_extraction()` will download +/- 800 MB of data into the `features/` directory.

In [5]:
helpers.download_data()
prepare_feature_extraction()

Access denied with the following error:



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/u/1/uc?id=1G6CtoMJEic_eyNJ0ffRKJ8JiR19tD0tb&export=download 



Data was downloaded.
Preparing feature extraction by downloading 4 files:
        
 ../sherlock/features/glove.6B.50d.txt, 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy,
        
 ../sherlock/features/par_vec_trained_400.pkl.trainables.syn1neg.npy, and 
 ../sherlock/features/par_vec_trained_400.pkl.wv.vectors.npy.
        
All files for extracting word and paragraph embeddings are present.


In [6]:
if not os.path.exists('../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy'):
    raise SystemExit(
        """
        Trained paragraph vectors do not exist,
        please run the '01-train-paragraph-vector-features' notebook before continuing
        """
    )

## Read in raw data
You can skip this step if you want to use a preprocessed data file.

## Report memory usage (can be slow)

In [7]:
report_memory = False

## Extract features
It is important that the string-representations of lists are first converted into lists of strings.
The labels should be a list of semantic types.

In [8]:
timestr = time.strftime("%Y%m%d-%H%M%S")

# Features will be output to the following files
X_test_filename_csv = f'../data/data/processed/test_{timestr}.csv'
X_train_filename_csv = f'../data/data/processed/train_{timestr}.csv'
X_validation_filename_csv = f'../data/data/processed/validation_{timestr}.csv'

### PREPARATION

In [9]:
# ensure embedding initialisation is outside of timing for extract_features
prepare_feature_extraction()
initialise_word_embeddings()
initialise_pretrained_model(400)
initialise_nltk()

Preparing feature extraction by downloading 4 files:
        
 ../sherlock/features/glove.6B.50d.txt, 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy,
        
 ../sherlock/features/par_vec_trained_400.pkl.trainables.syn1neg.npy, and 
 ../sherlock/features/par_vec_trained_400.pkl.wv.vectors.npy.
        
All files for extracting word and paragraph embeddings are present.
Initialising word embeddings
Initialise Word Embeddings process took 0:00:04.778517 seconds.
Initialise Doc2Vec Model, 400 dim, process took 0:00:05.602115 seconds. (filename = ../sherlock/features/par_vec_trained_400.pkl)
Initialised NLTK, process took 0:00:00.209538 seconds.


[nltk_data] Downloading package punkt to /home/ritvikp/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ritvikp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
%load_ext line_profiler

In [11]:
# default process creation mode is changed in Python 3.8 to 'spawn' which causes "name not defined" type errors. Force fork 'mode' for now (this only needs to be called once).
# https://bugs.python.org/issue39931
#mp.set_start_method('fork', force=True)

## EXTRACT FEATURES TO CSV

### TEST SET

In [12]:
values = load_parquet_values("../data/data/raw/test_values.parquet")

extract_features_to_csv(X_test_filename_csv, values)

values = None

Starting ../data/data/processed/test_20221128-062633.csv at 2022-11-28 06:26:44.533537. Rows=9932, using 128 CPU cores
Exporting 1588 column features
Finished. Processed 9932 rows in 0:00:07.346608, key_count=97


In [13]:
print(f'Finished at {datetime.now()}')

Finished at 2022-11-28 06:26:51.903506


### TRAIN SET

In [14]:
values = load_parquet_values("../data/data/raw/train_values.parquet")

extract_features_to_csv(X_train_filename_csv, values)

values = None

Starting ../data/data/processed/train_20221128-062633.csv at 2022-11-28 06:26:52.159259. Rows=52965, using 128 CPU cores
Exporting 1588 column features
Finished. Processed 52965 rows in 0:00:26.104362, key_count=128


In [15]:
print(f'Finished at {datetime.now()}')

Finished at 2022-11-28 06:27:18.338618


### VALIDATION SET

In [16]:
values = load_parquet_values("../data/data/raw/val_values.parquet")

extract_features_to_csv(X_validation_filename_csv, values)

values = None

Starting ../data/data/processed/validation_20221128-062633.csv at 2022-11-28 06:27:18.421261. Rows=3310, using 128 CPU cores
Exporting 1588 column features
Finished. Processed 3310 rows in 0:00:05.354544, key_count=34


In [17]:
print(f'Finished at {datetime.now()}')

Finished at 2022-11-28 06:27:24.071723


### Read Locally Processed Features

In [18]:
start = datetime.now()

X_test = pd.read_csv(X_test_filename_csv, dtype=np.float32)

print(f'Load Features (test) process took {datetime.now() - start} seconds.')

Load Features (test) process took 0:00:01.204271 seconds.


In [19]:
X_test.head()

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,1.0,1.714290,1.061220,1.0,4.0,1.0,12.0,0.596154,1.376080,...,-0.281380,-0.140214,-0.140021,0.042437,-0.152759,-0.136358,0.175482,-0.014018,0.005394,-0.422263
1,1.0,0.0,0.300000,0.210000,0.0,1.0,0.0,3.0,-1.238095,0.872872,...,-0.125739,0.020457,-0.284268,0.167863,-0.214490,-0.095541,0.059021,0.084747,-0.002000,-0.267915
2,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.028927,-0.015423,0.370082,0.074189,-0.226176,0.035250,-0.145200,0.116925,-0.087567,-0.322178
3,1.0,0.0,0.600000,0.640000,0.0,2.0,0.0,6.0,-0.921875,0.843750,...,-0.106598,-0.262913,0.209793,-0.178401,-0.087042,0.093397,-0.011860,-0.042827,-0.111934,-0.315480
4,1.0,0.0,0.047619,0.045351,0.0,1.0,0.0,1.0,16.049999,4.248529,...,-0.425667,-0.108041,0.103747,0.794387,-0.575408,-0.152921,-0.056353,-0.103970,-0.057168,-0.572790


In [20]:
start = datetime.now()

X_train = pd.read_csv(X_train_filename_csv, dtype=np.float32)

print(f'Load Features (train) process took {datetime.now() - start} seconds.')

Load Features (train) process took 0:00:05.960522 seconds.


In [21]:
X_train.head()

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.50,0.2500,0.0,1.0,0.5,1.0,-2.000000,0.000000,...,-0.008271,-0.012103,-0.005172,0.059884,-0.066609,-0.005661,0.050756,-0.048523,-0.008957,-0.054247
1,1.0,0.0,0.20,0.1600,0.0,1.0,0.0,5.0,0.250000,1.500000,...,-0.426114,-0.142305,0.244577,0.734353,-0.445607,-0.391222,-0.091195,-0.283493,-0.089952,-0.527054
2,0.0,0.0,0.00,0.0000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.082587,-0.006445,-0.016225,0.007082,-0.079259,-0.042428,-0.016068,0.013284,-0.068572,-0.131835
3,1.0,0.0,0.04,0.0384,0.0,1.0,0.0,1.0,20.041666,4.694855,...,-0.411747,-0.004611,-0.020559,0.759918,-0.645122,-0.268444,-0.378818,-0.248283,-0.095817,-0.904903
4,0.0,0.0,0.00,0.0000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,0.007048,-0.023109,-0.069917,0.086889,-0.132284,-0.098877,0.057632,-0.121708,-0.039745,-0.078869


In [22]:
start = datetime.now()

X_validation = pd.read_csv(X_validation_filename_csv, dtype=np.float32)

print(f'Load Features (validation) process took {datetime.now() - start} seconds.')

Load Features (validation) process took 0:00:00.417902 seconds.


In [23]:
X_validation.head()

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.300000,0.210000,0.0,1.0,0.0,6.0,-1.238100,0.872872,...,0.199000,-0.110489,0.262272,-0.041161,-0.169114,0.173513,0.267047,0.367141,0.332911,-0.506773
1,1.0,0.0,0.500000,0.250000,0.0,1.0,0.5,1.0,-2.000000,0.000000,...,-0.011504,-0.009263,-0.039588,0.038813,-0.023516,-0.013927,0.023792,-0.071949,0.004747,-0.006529
2,1.0,0.0,0.084337,0.077225,0.0,1.0,0.0,7.0,6.949248,2.991529,...,-0.979940,0.532293,0.145869,-0.091688,-0.335776,0.155618,-0.068057,0.203684,-0.010893,-0.515985
3,1.0,1.0,1.333333,0.222222,1.0,2.0,1.0,12.0,-1.500000,0.707107,...,-0.015089,-0.356702,0.230367,0.029790,-0.386576,-0.047297,-0.241256,-0.072495,0.192543,-0.362939
4,1.0,0.0,0.200000,0.160000,0.0,1.0,0.0,1.0,0.250000,1.500000,...,-0.073229,-0.141090,0.019915,0.036778,-0.147867,-0.192812,-0.191583,0.049382,0.030387,-0.196487


## Impute NaN values with feature means

In [24]:
start = datetime.now()

train_columns_means = pd.DataFrame(X_train.mean()).transpose()

print(f'Transpose process took {datetime.now() - start} seconds.')

Transpose process took 0:00:00.176170 seconds.


In [25]:
start = datetime.now()

X_train.fillna(train_columns_means.iloc[0], inplace=True)
X_validation.fillna(train_columns_means.iloc[0], inplace=True)
X_test.fillna(train_columns_means.iloc[0], inplace=True)

train_columns_means=None

print(f'FillNA process took {datetime.now() - start} seconds.')

FillNA process took 0:00:00.606861 seconds.


In [26]:
start = datetime.now()

X_train.to_parquet('../data/data/processed/train.parquet', engine='pyarrow', compression='snappy')
X_validation.to_parquet('../data/data/processed/validation.parquet', engine='pyarrow', compression='snappy')
X_test.to_parquet('../data/data/processed/test.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process took {datetime.now() - start} seconds.')

Save parquet process took 0:00:03.798439 seconds.


In [27]:
print(f'Completed at {datetime.now()}.')

Completed at 2022-11-28 06:27:36.999059.
